# AVANZADO. Extracción de Metadata para una mejor indexación y comprensión de un objeto Document en LlamaIndex

https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_metadata_extractor/

En muchos casos, especialmente cuando tenemos que tratar con documentos largos, un chunk o fragmento de texto puede caracer del contexto necesario para eliminar la ambigüedad entre ese fragmento y otros del mismo Document.

Un método para abordar esto es etiquetar manualmente cada fragmento de nuestro conjunto de datos. No obstante, hay que tener en cuenta que esto puede requerir mucho tiempo si tratamos con una gran cantidad de documentos o documentos que se actualizan de manera constante.

Para solucionarlo, utilizamos LLMs para extraer cierta información contextual relevante para el documento y así ayudar con la recuperación de la información de manera más efectiva.

Se utiliza el módulo Metadata Extractor de LlamaIndex.

In [1]:
# # Tienes que tener instalados los siguientes paquetes

# %pip install llama-index-llms-openai
# %pip install llama-index-extractors-entity

Si no has ejecutado nunca LlamaIndex también tendrás que instalarlo:

In [2]:
# %pip install llama-index

Luego ejecuta este código:

In [4]:
import nest_asyncio
nest_asyncio.apply()

El módulo nest_asyncio se utiliza para permitir el uso de bucles de eventos asyncio anidados en un entorno Jupyter Notebook o en Google Colab. El módulo proporciona un marco para escribir código concurrente utilizando la sintaxis async/await y evitar errores de lógica inesperados.

Luego sigue con esto:

In [5]:
import os
import openai

# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"
# O puedes utilizar la funcion load_dotenv del módulo dotenv, pero tendrás que tener creado un archivo .env con la clave OPENAI_API_KEY

from dotenv import load_dotenv
load_dotenv()

from llama_index.llms.openai import OpenAI
from llama_index.core.schema import MetadataMode

llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo", max_tokens=512)

Ahora vamos a crear un analizador de nodos que extraerá el título del documento y hipotéticas preguntas relacionadas con el contenido de ese chunk. Instanciamos diferentes clases del módulo extractors como SummartyExtractor y KeywordExtractor, aunque también puedes crear el tuyo utilizando la clase BaseExtractor.

In [6]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    BaseExtractor,
)
from llama_index.extractors.entity import EntityExtractor
from llama_index.core.node_parser import TokenTextSplitter

text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=512, chunk_overlap=128
)

class CustomExtractor(BaseExtractor):
    def extract(self, nodes):
        metadata_list = [
            {
                "custom": (
                    node.metadata["document_title"]
                    + "\n"
                    + node.metadata["excerpt_keywords"]
                )
            }
            for node in nodes
        ]
        return metadata_list


extractors = [
    TitleExtractor(nodes=5, llm=llm),
    QuestionsAnsweredExtractor(questions=3, llm=llm),
    # EntityExtractor(prediction_threshold=0.5),
    # SummaryExtractor(summaries=["prev", "self"], llm=llm),
    # KeywordExtractor(keywords=10, llm=llm),
    # CustomExtractor()
]

transformations = [text_splitter] + extractors

Utilizando la fuente de datos de Tesla que introducíamos en este artículo: https://www.codigollm.es/document-llamaindex-que-son-estos-objetos-en-el-contexto-de-los-modelos-llm/, y una vez cargada la información utilizando SimpleDirectoryReader, vamos a aplicar las transformaciones a los documentos que se han generado:

In [7]:
from llama_index.core import SimpleDirectoryReader

# Note the uninformative document file name, which may be a common scenario in a production setting
# tesla_docs = SimpleDirectoryReader(input_files=["data/TSLA-Q1-2024-Update.pdf"]).load_data()
tesla_docs = SimpleDirectoryReader(input_files=["data/TSLA-Q1-2024-Update.pdf"]).load_data()

# Nuestro Document tiene un total de 31 objetos
# tesla_front_pages = tesla_docs[0:3]
# tesla_content = tesla_docs[3:31]
# tesla_docs = tesla_front_pages + tesla_content

from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(transformations=transformations)

tesla_nodes = pipeline.run(documents=tesla_docs)

100%|██████████| 45/45 [00:19<00:00,  2.32it/s]


Para ver el resultado mejor vamos a utilizar una función auxiliar para ayudarnos a serializar, es decir, traducir una estructura compleja a un formato que Python pueda manejar, y luego imprimir el resultado más legible:

In [8]:
import json

def serialize_and_print_text_node(text_node):
    # Convertir el objeto TextNode a un diccionario
    text_node_dict = {
        'id_': text_node.id_,
        'embedding': text_node.embedding,
        'metadata': text_node.metadata,
        'excluded_embed_metadata_keys': text_node.excluded_embed_metadata_keys,
        'excluded_llm_metadata_keys': text_node.excluded_llm_metadata_keys,
        'relationships': {
            str(relationship): {
                'node_id': related_node_info.node_id,
                'node_type': str(related_node_info.node_type),
                'metadata': related_node_info.metadata,
                'hash': related_node_info.hash
            }
            for relationship, related_node_info in text_node.relationships.items()
        },
        'text': text_node.text,
        'start_char_idx': text_node.start_char_idx,
        'end_char_idx': text_node.end_char_idx,
        'text_template': text_node.text_template,
        'metadata_template': text_node.metadata_template,
        'metadata_seperator': text_node.metadata_seperator
    }

    # Serializar el diccionario a JSON con indentación para una mejor legibilidad
    json_data = json.dumps(text_node_dict, indent=2)

    # Imprimir el JSON serializado
    print(json_data)

Si guardamos el primer objeto que se ha creado text_node = tesla_nodes[0] y lo mostramos a través de la función serialize_and_print_text_node(text_node):

In [11]:
serialize_and_print_text_node(tesla_nodes[0])

{
  "id_": "6aee7193-074b-4758-8ed3-234847482740",
  "embedding": null,
  "metadata": {
    "page_label": "1",
    "file_name": "TSLA-Q1-2024-Update.pdf",
    "file_path": "data\\TSLA-Q1-2024-Update.pdf",
    "file_type": "application/pdf",
    "file_size": 7458951,
    "creation_date": "2024-06-16",
    "last_modified_date": "2024-06-16",
    "document_title": "Q1 2024 Update: A Comprehensive Overview of Candidate Titles and Content",
    "questions_this_excerpt_can_answer": "1. What is the title of the document that provides a comprehensive overview of candidate titles and content for the Q1 2024 update?\n2. When was the Q1 2024 update document created and last modified?\n3. What is the file size of the Q1 2024 update document in PDF format?"
  },
  "excluded_embed_metadata_keys": [
    "file_name",
    "file_type",
    "file_size",
    "creation_date",
    "last_modified_date",
    "last_accessed_date"
  ],
  "excluded_llm_metadata_keys": [
    "file_name",
    "file_type",
    "fil

Ahora, si algun método de estas clases no nos convence y queremos controlar más el output del modelo, podemos hacer una aproximación diferente utilizando también técnicas de data mining pero de manera más artesana y programando nuestra propia función:

In [12]:
import logging

# Configuración de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from openai import OpenAI

# Crear una instancia del cliente de OpenAI
client = OpenAI()

# Definimos una función para extraer las Entities del contenido
def get_entity_extraction_prompt(content):
    logger.info("Generando el prompt para la extracción de entidades.")
    try:
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Extrae las entidades más importantes del texto que te proporcionará el usuario. Identifica entidades como personas, organizaciones, ubicaciones, fechas, cantidades, etc. Devuelve las entidades como una lista separada por comas, así: John Smith, Microsoft, Nueva York, 2023-05-20, $1 millón. Devuelve un máximo de 10 entidades. Si no se encuentran entidades poner: ' '"},
                {"role": "user", "content": f"{content}"}
            ]
        )
        logger.info("Extracción de entidades completada con éxito.")
        return completion.choices[0].message.content
    except Exception as e:
        logger.error("Error al generar la extracción de entidades: %s", e)
        return None

Como queremos extraer las Entites de cada chunk hacemos una llamada al modelo con el prompt específico para cada Node o chunk del documento, así:

In [13]:
for node in tesla_nodes:
    text = node.get_text()
    try:
        entities = get_entity_extraction_prompt(text)
        logger.info("Entidades extraídas correctamente.")
        node.metadata["entities"] = entities.split(", ")
    except Exception as e:
        logger.error(f"Error al extraer entidades: {e}")
        node.metadata["entities"] = ["Error en la extracción"]
        
# Imprimir las entidades extraídas para cada nodo
for node in tesla_nodes:
    print(f"Nodo: {node.node_id}")
    print(f"Entidades: {node.metadata['entities']}")
    print("---")

INFO:__main__:Generando el prompt para la extracción de entidades.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Extracción de entidades completada con éxito.
INFO:__main__:Entidades extraídas correctamente.
INFO:__main__:Generando el prompt para la extracción de entidades.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Extracción de entidades completada con éxito.
INFO:__main__:Entidades extraídas correctamente.
INFO:__main__:Generando el prompt para la extracción de entidades.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Extracción de entidades completada con éxito.
INFO:__main__:Entidades extraídas correctamente.
INFO:__main__:Generando el prompt para la extracción de entidades.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Extracción de entidades com

Nodo: 6aee7193-074b-4758-8ed3-234847482740
Entidades: ['- 2024']
---
Nodo: ee07fe10-5eb9-4228-b2ad-83dfa67ba5fe
Entidades: ["  ' '"]
---
Nodo: 5c14b804-2cbe-4062-947d-3267523279c6
Entidades: ['Red Sea', 'Gigafactory Berlin', 'Fremont', 'Global EV', 'COGS', 'EV adoption', 'Cybertruck', 'regulatory credits', 'AI infrastructure', 'Supercharger.']
---
Nodo: 80f75ded-6c2f-4999-8c61-b7e9069c6fa4
Entidades: ['COGS', 'factories', 'production lines', 'AI', 'FSD', 'Supervised', 'subscription', 'price', 'cash flow', 'Free cash flow', 'AI infrastructure', 'capex', 'cash', 'investments.']
---
Nodo: ae92be14-1416-4941-aeb8-9768ab80e5ee
Entidades: ['Total automotive revenues', 'Energy generation and storage revenue', 'Services and other revenue', 'Total revenues', 'Total gross profit', 'Total GAAP gross margin', 'Operating expenses', 'Income from operations', 'Adjusted EBITDA', 'Net income.']
---
Nodo: 36461cd3-e483-4fc4-b3d1-70be6fd92d6a
Entidades: ['-674%', '22,402', '23,075', '26,077', '29,094', '

Y si volvemos a utilizar la función serialize_and_print_text_node(text_node) dónde text_node es text_node = tesla_nodes[1]:

In [16]:
serialize_and_print_text_node(tesla_nodes[2])

{
  "id_": "5c14b804-2cbe-4062-947d-3267523279c6",
  "embedding": null,
  "metadata": {
    "page_label": "3",
    "file_name": "TSLA-Q1-2024-Update.pdf",
    "file_path": "data\\TSLA-Q1-2024-Update.pdf",
    "file_type": "application/pdf",
    "file_size": 7458951,
    "creation_date": "2024-06-16",
    "last_modified_date": "2024-06-16",
    "document_title": "\"Accelerating Towards Profitable Growth: Tesla's Q1 Performance, Cost Reduction, Scaled Autonomy, and Record Production\"",
    "questions_this_excerpt_can_answer": "1. How much did Tesla invest in capital expenditures in Q1 of 2024?\n2. What challenges did Tesla face in Q1 of 2024, impacting their operations and profitability?\n3. How has Tesla been working towards increasing EV adoption and supporting their growth through vehicle financing programs and cost-cutting exercises in Q1 of 2024?",
    "entities": [
      "Red Sea",
      "Gigafactory Berlin",
      "Fremont",
      "Global EV",
      "COGS",
      "EV adoption",
 